<a href="https://colab.research.google.com/github/kskouegbe/21-Deep-Learning/blob/main/charity_funding_success_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [21]:
# import our dependencies
!pip install pyyaml h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Preprocessing

In [22]:
#  import and read the charity_data.csv
url = 'https://raw.githubusercontent.com/kskouegbe/21-Deep-Learning/main/Resources/charity_data.csv'
df = pd.read_csv(url)
df_len = len(df.index)
print(df_len)
df.head()

34299


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [23]:
# drop the non-beneficial ID columns, 'EIN' and 'NAME'
df.drop(['EIN', 'NAME'], axis=1, inplace=True)
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [25]:
# Look at APPLICATION_TYPE value counts for binning
app_values_s = df['APPLICATION_TYPE'].value_counts()
app_values_s

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [26]:
# Compile smaller vars into "other"
application_types_to_replace = []
for i, v in app_values_s.iteritems():
  if v < app_values_s[7]: #anything outside of top 7 is grouped into other
    application_types_to_replace.append(i)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [27]:
# Look at CLASSIFICATION value counts for binning
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [28]:
# looking at CLASSIFICATION value counts >1
bool_s = df['CLASSIFICATION'].value_counts()>1
class_value_s = df['CLASSIFICATION'].value_counts()[bool_s]
class_value_s

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [29]:
# dropping rows where classification < 2
class_1 = df['CLASSIFICATION'].value_counts()[df['CLASSIFICATION'].value_counts()<2].index.to_list()
df = df[df['CLASSIFICATION'].isin(class_1) == False]
print(f'Was {df_len}, dropped {df_len - len(df.index)}, now {len(df.index)}')

Was 34299, dropped 26, now 34273


In [30]:
# Assign cutoff value and create a list of classifications to be replaced
classifications_to_replace = []
for i, v in class_value_s.iteritems():
  if v < class_value_s[7]: #anything outside of top 7 is grouped into other
    classifications_to_replace.append(i)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      977
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [31]:
# convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df)
df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [32]:
# Split our preprocessed data into our features and target arrays
X = df_dummies.drop('IS_SUCCESSFUL', axis=1)
y = df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) #because 42 is the only random state we live by

In [33]:
# Create a StandardScaler instances
#assuming data is normal, hence use standardscalar
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# Compile, Train, Eval Model

In [34]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 6
hidden_nodes_layer2 = 6

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 282       
                                                                 
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model and train over more than 100 epochs
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6212 - accuracy: 0.6787
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5800 - accuracy: 0.7102
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5715 - accuracy: 0.7227
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5678 - accuracy: 0.7233
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5611 - accuracy: 0.7248
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5580 - accuracy: 0.7246
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5564 - accuracy: 0.7258
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5556 - accuracy: 0.7258
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5551 - accuracy: 0.7262
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5545 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5437 - accuracy: 0.7387 - 391ms/epoch - 1ms/step
Loss: 0.543691873550415, Accuracy: 0.7387093305587769


In [37]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharityfinal.h5')